In [1]:
import json, jsonschema, jwt, datetime, uuid, requests

## Utility functions

In [2]:
def urn_uuid(uuid):
    return f'urn:uuid:{uuid}'

In [3]:
class MockKeyManager:

    def __init__(self):
        self.secrets = {}

    def add(self, key):
        kid = urn_uuid(uuid.uuid4())
        self.secrets[kid] = key
        return kid

    def get(self, kid):
        if kid in self.secrets:
            return self.secrets[kid]
        else:
            return None

    def delete(self, kid):
        if kid in self.secrets:
            del self.secrets[kid]
        else:
            return None

    def reset(self):
        self.secrets = {}

    def list(self):
        for kid in self.secrets:
            print(f'{kid}: {self.secrets[kid]}')

In [4]:
def sign(key_manager, credential, key, verifier):
    kid = key_manager.add(key)
    headers = {
    "alg": "HS256",
    "typ": "JWT",
    "kid": kid  # Custom field (not standard)
    }
    payload = {
        "iss": credential["issuer"],  # Issuer
        "aud": verifier,  # Audience
        "exp": datetime.datetime.utcnow() + datetime.timedelta(days=1),  # Expiration time
        "nbf": datetime.datetime.utcnow(),  # Not before
        "iat": datetime.datetime.utcnow(),  # Issued at
        "kid": kid,
    }
    if "VerifiableCredential" in credential["type"]:
        payload["sub"] = credential["credentialSubject"]["id"]
        payload["vc"] = credential
    elif "VerifiablePresentation" in credential["type"]:
        payload["vp"] = credential
    return jwt.encode(payload, key, algorithm='HS256', headers=headers)

In [5]:
def verify(key_manager, signed_credential, verifier):
    try:
        key = key_manager.get(jwt.get_unverified_header(signed_credential)["kid"])
        decoded_payload = jwt.decode(signed_credential, key, algorithms='HS256', audience=verifier)
        if 'vp' in decoded_payload:
            decoded_payload["vp"]["verifiableCredential"] = [ 
                verify(key_manager, payload, verifier) for payload in decoded_payload["vp"]["verifiableCredential"] 
            ]
            return decoded_payload['vp']
        else:
            return decoded_payload['vc']
    except Exception as e:
        print(f'Verification error: {e}')
        return None

In [6]:
KEY_MANAGER = MockKeyManager()
VERIFIER = "https://verifier.example.gov"

## A example Merit credential

In [7]:
merit = json.load(open("test_merit.json", "r"))
merit

{'id': 'dc30880d-2d4b-447c-905e-b79075aea95b',
 'acceptedAt': '2023-07-17T08:52:19.489566Z',
 'active': True,
 'activenessFailures': None,
 'authorizedAt': '2023-07-17T08:52:15.960677Z',
 'completed': True,
 'completenessFailures': None,
 'createdAt': '2023-07-17T15:52:04.3018Z',
 'description': 'Template for admin merit',
 'fields': [{'canShare': False,
   'description': "Use this to store recipient's first name",
   'fieldKind': {'fieldType': 'Text',
    'id': 'a9fa2832-4107-4c95-b7d8-0cdd15a7e2bc',
    'name': 'Text'},
   'name': 'First Name',
   'templateFieldID': '1b397a27-14a0-4d43-a36f-6006611f4073',
   'templateFieldLineage': ['3dc3f769-70cc-4e7e-81cc-ed103f9eb601'],
   'updatedAt': '2023-07-17T15:52:19.496539Z',
   'validationErrors': None,
   'value': 'Hank'},
  {'canShare': False,
   'description': "Use this to store recipient's last name",
   'fieldKind': {'fieldType': 'Text',
    'id': 'a9fa2832-4107-4c95-b7d8-0cdd15a7e2bc',
    'name': 'Text'},
   'name': 'Last Name',
   

## A verifiable credential generated from the example Merit credential

In [8]:
id = urn_uuid(merit["id"])
issuer = urn_uuid(merit["issuer"]["id"])
subject = urn_uuid(merit["recipient"]["id"])

verifiable_credential = {
    "@context": [
        "https://www.w3.org/2018/credentials/v2",
        "https://www.w3.org/2018/credentials/examples/v2",
        "https://schema.org/"
    ],
    "issuer": issuer,
    "id": id,
    "type": ["VerifiableCredential", merit["name"]],
    "credentialSubject": {
        "id": subject,
        "merit": { field["name"]: field["value"] for field in merit["fields"] },
    },
    "issuanceDate": merit["createdAt"],
    "credentialStatus": {
        "id": f'https://vc.merit.com/status/{requests.utils.quote(id)}',
        "type": "MeritCredentialStatusList2024",
        "statusPurpose": merit['state']['name'],
        "occurredAt": merit['state']['occurredAt']
    }
}
verifiable_credential

{'@context': ['https://www.w3.org/2018/credentials/v2',
  'https://www.w3.org/2018/credentials/examples/v2',
  'https://schema.org/'],
 'issuer': 'urn:uuid:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
 'id': 'urn:uuid:dc30880d-2d4b-447c-905e-b79075aea95b',
 'type': ['VerifiableCredential', 'Admin merit'],
 'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'merit': {'First Name': 'Hank',
   'Last Name': 'Lindgren',
   'Email': 'oleg+user2@gomerits.com',
   'Issuing Org Name': 'Merit International, Inc.',
   'Org UUID': 'user2',
   'Admin Phone Number': '6317070926',
   'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd',
   'Org Name': 'Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb',
   'Org Description': 'This is very important organization',
   'Org Governing Country': 'US',
   'Org Governing State': 'CA'}},
 'issuanceDate': '2023-07-17T15:52:04.3018Z',
 'credentialStatus': {'id': 'https://vc.merit.com/status/urn%3Auuid%3Adc30880d-2d

## JSON Schema validation of the verifiable credential

In [22]:
vc_schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "@context": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "minItems": 1
        },
        "id": {
            "type": "string"
        },
        "type": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "contains": {
                "const": "VerifiableCredential"
            }
        },
        "issuer": {
            "type": "string"
        },
        "issuanceDate": {
            "type": "string",
            "format": "date-time"
        },
        "credentialSubject": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string"
                }
            },
            "required": ["id"]
        },
        "credentialStatus": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string"
                },
                "type": {
                    "type": "string"
                },
                "statusPurpose": {
                    "type": "string"
                },
                "occurredAt": {
                    "type": "string",
                    "format": "date-time"
                }
            },
            "required": ["id"]
        },
        "proof": {
            "type": "object",
            "properties": {
                "type": {
                    "type": "string"
                },
                "created": {
                    "type": "string",
                    "format": "date-time"
                },
                "proofValue": {
                    "type": "string"
                },
                "verificationMethod": {
                    "type": "string"
                },
                "jws": {
                    "type": "string"
                }
            },
            "required": ["type", "created", "proofValue"]
        }
    },
    "required": ["@context", "id", "type", "issuer", "issuanceDate", "credentialSubject", "credentialStatus"]
}

In [23]:
try:
    jsonschema.validate(instance=verifiable_credential, schema=vc_schema)
    print("Credential is valid.")
except jsonschema.exceptions.ValidationError as e:
    print("Credential is not valid:", e)

Credential is valid.


## JSON Web Token (JWT) serialization of the verifiable credential

In [11]:
signed_vc = sign(KEY_MANAGER, verifiable_credential, "secret", VERIFIER)
signed_vc

'eyJhbGciOiJIUzI1NiIsImtpZCI6InVybjp1dWlkOmVlMDNiYjg1LTU0MjAtNDMzMi1iYTY0LTBlM2JkY2RlM2VmNCIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJ1cm46dXVpZDo5YTVlYjkyYS1iMzRhLTQyYjktOGZiNy0xMGNlOTQ0OWU4M2UiLCJhdWQiOiJodHRwczovL3ZlcmlmaWVyLmV4YW1wbGUuZ292IiwiZXhwIjoxNjk0OTg3MTczLCJuYmYiOjE2OTQ5MDA3NzMsImlhdCI6MTY5NDkwMDc3Mywia2lkIjoidXJuOnV1aWQ6ZWUwM2JiODUtNTQyMC00MzMyLWJhNjQtMGUzYmRjZGUzZWY0Iiwic3ViIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjIiLCJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy9leGFtcGxlcy92MiIsImh0dHBzOi8vc2NoZW1hLm9yZy8iXSwiaXNzdWVyIjoidXJuOnV1aWQ6OWE1ZWI5MmEtYjM0YS00MmI5LThmYjctMTBjZTk0NDllODNlIiwiaWQiOiJ1cm46dXVpZDpkYzMwODgwZC0yZDRiLTQ0N2MtOTA1ZS1iNzkwNzVhZWE5NWIiLCJ0eXBlIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiQWRtaW4gbWVyaXQiXSwiY3JlZGVudGlhbFN1YmplY3QiOnsiaWQiOiJ1cm46dXVpZDoxOTZhOGM4Zi02ZmQ5LTQ1NmItODE5My00MTU5YjhkYzNhYjYiLCJtZXJpdCI6eyJGaXJzdCBOYW1lIjoiSGFuayIsIkxhc3QgTmFtZSI6IkxpbmRncmVuIiwiRW

In [12]:
key = KEY_MANAGER.get(jwt.get_unverified_header(signed_vc)["kid"])
verifiable_credential = verify(KEY_MANAGER, signed_vc, VERIFIER)
verifiable_credential

{'@context': ['https://www.w3.org/2018/credentials/v2',
  'https://www.w3.org/2018/credentials/examples/v2',
  'https://schema.org/'],
 'issuer': 'urn:uuid:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
 'id': 'urn:uuid:dc30880d-2d4b-447c-905e-b79075aea95b',
 'type': ['VerifiableCredential', 'Admin merit'],
 'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'merit': {'First Name': 'Hank',
   'Last Name': 'Lindgren',
   'Email': 'oleg+user2@gomerits.com',
   'Issuing Org Name': 'Merit International, Inc.',
   'Org UUID': 'user2',
   'Admin Phone Number': '6317070926',
   'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd',
   'Org Name': 'Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb',
   'Org Description': 'This is very important organization',
   'Org Governing Country': 'US',
   'Org Governing State': 'CA'}},
 'issuanceDate': '2023-07-17T15:52:04.3018Z',
 'credentialStatus': {'id': 'https://vc.merit.com/status/urn%3Auuid%3Adc30880d-2d

## Adding geofencing into the credentialSubject of the verifiable credential

In [13]:
verifiable_credential["credentialSubject"]["eligibleRegion"] = {
    "type": "GeoCircle",
    "geoMidpoint": {
        "type": "GeoCoordinates",
        "latitude": "38.88508",
        "longitude": "-77.02295"
    },
    "geoRadius": "50"
}
verifiable_credential

{'@context': ['https://www.w3.org/2018/credentials/v2',
  'https://www.w3.org/2018/credentials/examples/v2',
  'https://schema.org/'],
 'issuer': 'urn:uuid:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
 'id': 'urn:uuid:dc30880d-2d4b-447c-905e-b79075aea95b',
 'type': ['VerifiableCredential', 'Admin merit'],
 'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'merit': {'First Name': 'Hank',
   'Last Name': 'Lindgren',
   'Email': 'oleg+user2@gomerits.com',
   'Issuing Org Name': 'Merit International, Inc.',
   'Org UUID': 'user2',
   'Admin Phone Number': '6317070926',
   'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd',
   'Org Name': 'Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb',
   'Org Description': 'This is very important organization',
   'Org Governing Country': 'US',
   'Org Governing State': 'CA'},
  'eligibleRegion': {'type': 'GeoCircle',
   'geoMidpoint': {'type': 'GeoCoordinates',
    'latitude': '38.88508',
    'longitud

In [14]:
signed_geofenced_vc = sign(KEY_MANAGER, verifiable_credential, "secret2", VERIFIER)
signed_geofenced_vc

'eyJhbGciOiJIUzI1NiIsImtpZCI6InVybjp1dWlkOmIwYzU5NDU1LWNiOTAtNGIyZC05MzZiLTU1MzdhODYwNmY2OSIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJ1cm46dXVpZDo5YTVlYjkyYS1iMzRhLTQyYjktOGZiNy0xMGNlOTQ0OWU4M2UiLCJhdWQiOiJodHRwczovL3ZlcmlmaWVyLmV4YW1wbGUuZ292IiwiZXhwIjoxNjk0OTg3MTczLCJuYmYiOjE2OTQ5MDA3NzMsImlhdCI6MTY5NDkwMDc3Mywia2lkIjoidXJuOnV1aWQ6YjBjNTk0NTUtY2I5MC00YjJkLTkzNmItNTUzN2E4NjA2ZjY5Iiwic3ViIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjIiLCJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy9leGFtcGxlcy92MiIsImh0dHBzOi8vc2NoZW1hLm9yZy8iXSwiaXNzdWVyIjoidXJuOnV1aWQ6OWE1ZWI5MmEtYjM0YS00MmI5LThmYjctMTBjZTk0NDllODNlIiwiaWQiOiJ1cm46dXVpZDpkYzMwODgwZC0yZDRiLTQ0N2MtOTA1ZS1iNzkwNzVhZWE5NWIiLCJ0eXBlIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiQWRtaW4gbWVyaXQiXSwiY3JlZGVudGlhbFN1YmplY3QiOnsiaWQiOiJ1cm46dXVpZDoxOTZhOGM4Zi02ZmQ5LTQ1NmItODE5My00MTU5YjhkYzNhYjYiLCJtZXJpdCI6eyJGaXJzdCBOYW1lIjoiSGFuayIsIkxhc3QgTmFtZSI6IkxpbmRncmVuIiwiRW

## Generating a geolocation verifiable credential

In [15]:
geolocation_vc_id = urn_uuid(uuid.uuid4())
geolocation_vc = {
    '@context': [
        'https://www.w3.org/2018/credentials/v1', 
        'https://www.w3.org/2018/credentials/examples/v1', 
        'https://schema.org/'
    ],
    'issuer': subject,
    'id': geolocation_vc_id,
    'type': ['VerifiableCredential', 'RealtimeGeolocation'],
    'credentialSubject' : {
        'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
        'geo': {
            'type': 'GeoCoordinates',
            'latitude': '38.88460',
            'longitude': '-77.02273'
        }
    },
    'issuanceDate': datetime.datetime.utcnow().isoformat() + 'Z'
}
geolocation_vc

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
 'id': 'urn:uuid:3bf38946-c841-48ec-b39c-e8c2edf894eb',
 'type': ['VerifiableCredential', 'RealtimeGeolocation'],
 'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'geo': {'type': 'GeoCoordinates',
   'latitude': '38.88460',
   'longitude': '-77.02273'}},
 'issuanceDate': '2023-09-16T21:46:13.427991Z'}

In [16]:
signed_geolocation_vc = sign(KEY_MANAGER, geolocation_vc, "secret3", VERIFIER)
signed_geolocation_vc

'eyJhbGciOiJIUzI1NiIsImtpZCI6InVybjp1dWlkOjJhOTM1OTk0LWI2MjQtNDljOC05MWM1LWYwMTc0ODNjZTI0NyIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJ1cm46dXVpZDoxOTZhOGM4Zi02ZmQ5LTQ1NmItODE5My00MTU5YjhkYzNhYjYiLCJhdWQiOiJodHRwczovL3ZlcmlmaWVyLmV4YW1wbGUuZ292IiwiZXhwIjoxNjk0OTg3MTczLCJuYmYiOjE2OTQ5MDA3NzMsImlhdCI6MTY5NDkwMDc3Mywia2lkIjoidXJuOnV1aWQ6MmE5MzU5OTQtYjYyNC00OWM4LTkxYzUtZjAxNzQ4M2NlMjQ3Iiwic3ViIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiLCJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy9leGFtcGxlcy92MSIsImh0dHBzOi8vc2NoZW1hLm9yZy8iXSwiaXNzdWVyIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwiaWQiOiJ1cm46dXVpZDozYmYzODk0Ni1jODQxLTQ4ZWMtYjM5Yy1lOGMyZWRmODk0ZWIiLCJ0eXBlIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiUmVhbHRpbWVHZW9sb2NhdGlvbiJdLCJjcmVkZW50aWFsU3ViamVjdCI6eyJpZCI6InVybjp1dWlkOjE5NmE4YzhmLTZmZDktNDU2Yi04MTkzLTQxNTliOGRjM2FiNiIsImdlbyI6eyJ0eXBlIjoiR2VvQ29vcmRpbmF0ZXMiLCJsYXRpdHVkZSI6IjM4Lj

## Generating a verifiable presentation combining the geofenced credential with the geolocation credential

In [17]:
vp_id = urn_uuid(uuid.uuid4())
verifiable_presentation = {
    '@context': [
        'https://www.w3.org/2018/credentials/v1', 
        'https://www.w3.org/2018/credentials/examples/v1', 
        'https://schema.org/'
    ],
    'issuer': subject,
    'id': vp_id,
    'type': ['VerifiablePresentation'],
    'verifiableCredential': [
        signed_geofenced_vc,
        signed_geolocation_vc
    ]
}
verifiable_presentation

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
 'id': 'urn:uuid:c686e6c2-3564-4009-bd0e-f3eb26849650',
 'type': ['VerifiablePresentation'],
 'verifiableCredential': ['eyJhbGciOiJIUzI1NiIsImtpZCI6InVybjp1dWlkOmIwYzU5NDU1LWNiOTAtNGIyZC05MzZiLTU1MzdhODYwNmY2OSIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJ1cm46dXVpZDo5YTVlYjkyYS1iMzRhLTQyYjktOGZiNy0xMGNlOTQ0OWU4M2UiLCJhdWQiOiJodHRwczovL3ZlcmlmaWVyLmV4YW1wbGUuZ292IiwiZXhwIjoxNjk0OTg3MTczLCJuYmYiOjE2OTQ5MDA3NzMsImlhdCI6MTY5NDkwMDc3Mywia2lkIjoidXJuOnV1aWQ6YjBjNTk0NTUtY2I5MC00YjJkLTkzNmItNTUzN2E4NjA2ZjY5Iiwic3ViIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjIiLCJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy9leGFtcGxlcy92MiIsImh0dHBzOi8vc2NoZW1hLm9yZy8iXSwiaXNzdWVyIjoidXJuOnV1aWQ6OWE1ZWI5MmEtYjM0YS00MmI5LThmYjctMTBjZTk0

In [25]:
vp_schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "@context": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "minItems": 1
        },
        "id": {
            "type": "string"
        },
        "type": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "contains": {
                "const": "VerifiablePresentation"
            }
        },
        "issuer": {
            "type": "string"
        },
        "verifiableCredential": {
            "type": "array",
            "items": {
                "type": "string"
            }
        },
        "proof": {
            "type": "object",
            "properties": {
                "type": {
                    "type": "string"
                },
                "created": {
                    "type": "string",
                    "format": "date-time"
                },
                "proofValue": {
                    "type": "string"
                },
                "verificationMethod": {
                    "type": "string"
                },
                "jws": {
                    "type": "string"
                }
            },
            "required": ["type", "created", "proofValue"]
        }
    },
    "required": ["@context", "id", "type", "issuer", "verifiableCredential"]
}

In [26]:
try:
    jsonschema.validate(instance=verifiable_presentation, schema=vp_schema)
    print("Presentation is valid.")
except jsonschema.exceptions.ValidationError as e:
    print("Presentation is not valid:", e)

Presentation is valid.


In [18]:
signed_geoaware_vp = sign(KEY_MANAGER, verifiable_presentation, "secret1", VERIFIER)
signed_geoaware_vp

'eyJhbGciOiJIUzI1NiIsImtpZCI6InVybjp1dWlkOjNjNGJhYTdjLWI2OGUtNGYyOS04ZTZkLTUwM2JjYTllMTViYiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJ1cm46dXVpZDoxOTZhOGM4Zi02ZmQ5LTQ1NmItODE5My00MTU5YjhkYzNhYjYiLCJhdWQiOiJodHRwczovL3ZlcmlmaWVyLmV4YW1wbGUuZ292IiwiZXhwIjoxNjk0OTg3MTczLCJuYmYiOjE2OTQ5MDA3NzMsImlhdCI6MTY5NDkwMDc3Mywia2lkIjoidXJuOnV1aWQ6M2M0YmFhN2MtYjY4ZS00ZjI5LThlNmQtNTAzYmNhOWUxNWJiIiwidnAiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiLCJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy9leGFtcGxlcy92MSIsImh0dHBzOi8vc2NoZW1hLm9yZy8iXSwiaXNzdWVyIjoidXJuOnV1aWQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwiaWQiOiJ1cm46dXVpZDpjNjg2ZTZjMi0zNTY0LTQwMDktYmQwZS1mM2ViMjY4NDk2NTAiLCJ0eXBlIjpbIlZlcmlmaWFibGVQcmVzZW50YXRpb24iXSwidmVyaWZpYWJsZUNyZWRlbnRpYWwiOlsiZXlKaGJHY2lPaUpJVXpJMU5pSXNJbXRwWkNJNkluVnlianAxZFdsa09tSXdZelU1TkRVMUxXTmlPVEF0TkdJeVpDMDVNelppTFRVMU16ZGhPRFl3Tm1ZMk9TSXNJblI1Y0NJNklrcFhWQ0o5LmV5SnBjM01pT2lKMWNtNDZkWFZwWkRvNVlUVmxZamt5WVMxaU16UmhMVFF5WWprdE9HWmlOeTB4TUdObE

In [19]:
decoded_vp = verify(KEY_MANAGER, signed_geoaware_vp, VERIFIER)
decoded_vp

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
 'id': 'urn:uuid:c686e6c2-3564-4009-bd0e-f3eb26849650',
 'type': ['VerifiablePresentation'],
 'verifiableCredential': [{'@context': ['https://www.w3.org/2018/credentials/v2',
    'https://www.w3.org/2018/credentials/examples/v2',
    'https://schema.org/'],
   'issuer': 'urn:uuid:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
   'id': 'urn:uuid:dc30880d-2d4b-447c-905e-b79075aea95b',
   'type': ['VerifiableCredential', 'Admin merit'],
   'credentialSubject': {'id': 'urn:uuid:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
    'merit': {'First Name': 'Hank',
     'Last Name': 'Lindgren',
     'Email': 'oleg+user2@gomerits.com',
     'Issuing Org Name': 'Merit International, Inc.',
     'Org UUID': 'user2',
     'Admin Phone Number': '6317070926',
     'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b63